# Практическое задание №4

Данные data_big.csv о финских алкоголиках, переменные — признаки в первый день отказа от запоя:

Зависимая переменная - depressed.mood.1, остальные рассматриваются в качестве независимых. Независимые категориальные переменные упорядочить по значимости влияния на зависимую переменную. Привести соответствующие частоты, значимости критреия хи-квадрат и точного критерия Фишера. По совместному распределению первых двух наиболее значимых переменных вычисляете односторонний коэффициент неопределенности относительно зависимой переменной.

In [31]:
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt

In [32]:
import warnings
warnings.filterwarnings('ignore')

Считаем данные:

In [33]:
df = pd.read_csv("data_big.csv")

In [34]:
df.head(10)

,Unnamed: 0,X.1,depressed.mood.1,anxiety.1,suspiciousness.1,irritability.1,craving.to.alcohol.1,weakness.1,insomia.1,headache.1,...,combined2.9,HR.9,SBP.9,DBP.9,MBP.9,SV.9,CO.9,SI.9,CI.9,TPR.9
0,1,1,1,1,0,1,1,1,1,0,...,0,68.0,108.0,70.0,83.0,74.0,5.0,40.0,2.7,1321.0
1,2,2,1,1,0,0,1,1,2,1,...,9,63.0,114.0,70.0,85.0,123.0,7.7,60.0,3.8,879.0
2,3,3,1,1,0,0,0,2,1,0,...,0,64.0,120.0,80.0,93.0,106.0,6.8,55.0,3.5,1098.0
3,4,4,2,2,0,0,0,2,0,0,...,0,56.0,124.0,90.0,101.0,90.0,5.1,43.0,2.4,1600.0
4,5,5,1,1,0,0,2,2,1,0,...,0,66.0,116.0,78.0,90.0,90.0,5.9,45.0,2.2,1228.0
5,6,6,1,1,0,1,1,2,0,0,...,0,62.0,142.0,90.0,107.0,113.0,7.0,56.0,3.4,1225.0
6,7,7,1,1,0,1,1,2,2,1,...,0,85.0,110.0,84.0,93.0,46.0,3.9,21.0,1.8,1899.0
7,8,8,1,1,0,1,0,1,2,1,...,0,68.0,134.0,88.0,103.0,74.0,6.4,33.0,2.9,1290.0
8,9,9,1,1,0,0,2,1,0,0,...,0,76.0,134.0,76.0,96.0,116.0,8.6,60.0,4.4,886.0
9,10,10,1,1,0,1,1,1,2,1,...,0,64.0,110.0,70.0,83.0,43.0,2.7,27.0,1.7,2467.0


In [35]:
df = df[df.columns[2:20]]

In [36]:
#df = df.drop(columns=['suspiciousness.1', 'polyuria.1', 'diarrhea.1']);

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   depressed.mood.1            34 non-null     int64
 1   anxiety.1                   34 non-null     int64
 2   suspiciousness.1            34 non-null     int64
 3   irritability.1              34 non-null     int64
 4   craving.to.alcohol.1        34 non-null     int64
 5   weakness.1                  34 non-null     int64
 6   insomia.1                   34 non-null     int64
 7   headache.1                  34 non-null     int64
 8   tremor.1                    34 non-null     int64
 9   polyuria.1                  34 non-null     int64
 10  sweating.1                  34 non-null     int64
 11  transient.hallusinations.1  34 non-null     int64
 12  vomiting.1                  34 non-null     int64
 13  anoreksia.1                 34 non-null     int64
 14  diarrhea.1  

In [38]:
df.columns

Index(['depressed.mood.1', 'anxiety.1', 'suspiciousness.1', 'irritability.1',
       'craving.to.alcohol.1', 'weakness.1', 'insomia.1', 'headache.1',
       'tremor.1', 'polyuria.1', 'sweating.1', 'transient.hallusinations.1',
       'vomiting.1', 'anoreksia.1', 'diarrhea.1', 'thirst.1', 'chest.pain.1',
       'hyperemia.1'],
      dtype='object')

In [39]:
for col in df.columns[1:]:
    df[col] = [1 if value == 2 else value for value in df[col]]

Определим нужные функции для дальнейших вычислений:

In [40]:
def entropy(x):
    p = x/np.sum(x)
    p = p[p != 0]
    return - np.sum(p * np.log2(p))

In [41]:
def uncertian(tab):
    tab = np.array(tab)
    Hxy = entropy(tab.flatten())
    Hx = entropy(np.sum(tab, axis=1))
    Hy = entropy(np.sum(tab, axis=0))
    I = Hy + Hx - Hxy
    return I/Hx*100, I/Hy*100, 2*I/(Hx+Hy)*100

In [42]:
def chi2P(tab):
    tab = np.array(tab)
    r = np.sum(tab, axis=1)
    c = np.sum(tab, axis=0)
    chi2_ = 0

    for j in range(tab.shape[1]):
        for i in range(tab.shape[0]):
            chi2_ += (tab[i, j]**2) / (r[i] * c[j])

    chi2_ = (chi2_ - 1) * np.sum(tab)
    p_value = 1 - scipy.stats.chi2.cdf(chi2_, (tab.shape[0] - 1) * (tab.shape[1] - 1))

    return p_value

Выичслим метрики для каждой пары зависимой и независимой переменной

In [45]:
X = np.array(df["depressed.mood.1"])
print("Энтропия признака "+"depressed.mood.1"+f": {entropy(pd.DataFrame(X).value_counts())}")
print("\n")
sig = {}
for feature in df.columns[1:]:
    print("Название признака: " + feature)
    Y = np.array(df[feature])
    print(Y)
    tab = pd.crosstab(X, Y)
    #tab = tab.T
    print(tab)
    print(f"Условыне вероятности: {tab.iloc[:,1] / tab.sum(axis=1)}")
    res = scipy.stats.fisher_exact(tab)
    print(f"Точный критерий Фишера: {res[1]}")
    p_value = chi2P(tab)
    sig[feature] = p_value
    print(f"Точный критерий Пирсона: {p_value}")
    print("Энтропия признака "+feature+f": {entropy(pd.DataFrame(Y).value_counts())}")
    print(f"Коэффициенты неопределенности: {uncertian(tab)}")
    print("\n")

Энтропия признака depressed.mood.1: 0.6722948170756379


Название признака: anxiety.1
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1]
col_0  0   1
row_0       
1      3  25
2      2   4
Условыне вероятности: row_0
1    0.892857
2    0.666667
dtype: float64
Точный критерий Фишера: 0.20530015525271686
Точный критерий Пирсона: 0.15570662454844986
Энтропия признака anxiety.1: 0.6024308020404452
Коэффициенты неопределенности: (5.329536383640961, 5.947603734739486, 5.6216328194963)


Название признака: suspiciousness.1
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
col_0   0  1
row_0       
1      28  0
2       5  1
Условыне вероятности: row_0
1    0.000000
2    0.166667
dtype: float64
Точный критерий Фишера: 0.1764705882352941
Точный критерий Пирсона: 0.028326575664195386
Энтропия признака suspiciousness.1: 0.19143325481419343
Коэффициенты неопределенности: (11.412168255812407, 40.07841572471113, 17.765641339387685)


Название признака: irritabil

### Какие переменные наиболее значимые?

In [48]:
sorted(sig.items(), key=lambda x: x[1])

[('suspiciousness.1', 0.028326575664195386),
 ('tremor.1', 0.028326575664195386),
 ('hyperemia.1', 0.028326575664195386),
 ('sweating.1', 0.049601614355035295),
 ('vomiting.1', 0.09210313446999896),
 ('anxiety.1', 0.15570662454844986),
 ('diarrhea.1', 0.21603652619398117),
 ('chest.pain.1', 0.368261074002577),
 ('insomia.1', 0.39488069860207964),
 ('anoreksia.1', 0.4010856191936367),
 ('weakness.1', 0.4554344386045145),
 ('transient.hallusinations.1', 0.4554344386045145),
 ('thirst.1', 0.4554344386045145),
 ('headache.1', 0.5134639119071325),
 ('irritability.1', 0.7854126368894525),
 ('polyuria.1', 0.7934876018690161),
 ('craving.to.alcohol.1', 0.9118147754049294)]

In [53]:
tb = pd.crosstab(df["depressed.mood.1"], [df["suspiciousness.1"], df["tremor.1"]])

In [54]:
tb

suspiciousness.1  0      1
tremor.1          0   1  1
depressed.mood.1          
1                 0  28  0
2                 1   4  1

In [56]:
print(uncertian(tb))

(23.90391072577641, 42.11555826658952, 30.497868593496325)


**Выводы:**

- Если наблюдается подозрительность и тремор, то, скорее всего, есть и депрессивное настроение. Определяется 24% информации.
- Если наблюдается депрессивное настроение, то, скорее всего, есть подозрительность и тремор. Определяется 42% информации.